#### Ollama - llama3.2:3b 기반 LangChain 기본 모듈 사용해보기

- python==3.12
- ollama==0.15.4
- langchain==1.2.8
- langgraph==1.0.7
- langsmith==0.6.8

- ollama 설치
> https://ollama.com/

- 설치 이후 버전 확인
> ollama --version 

- LLM 다운로드 - 로컬환경 편하게 작동하도록, 다만 성능은 좋지않음
> ollama pull llama3.2:3b

- LangSmith API Key 발급 - 실행되는 과정 모니터링 가능
> https://smith.langchain.com/o/738f7d5d-9b23-4da4-819b-caac8931479b/settings/apikeys

---

#### 모델에게 Prompt (입력) 던지기 - (init_chat_model)

In [ ]:
""" LangSmith 트레이싱 - 세션 시작/재시작 후 맨 먼저 실행 """
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_*"  # smith.langchain.com → Settings → API Keys
os.environ["LANGCHAIN_PROJECT"] = "TEST"

In [2]:
import os
from langchain.chat_models import init_chat_model

#model, api_key, temperature, max_tokens, timeout, max_retries

model = init_chat_model(
    model="llama3.2:3b", 
    model_provider="ollama",
    temperature=0.8,
)
print(model, "\n")

response = model.invoke("Hello, Who Are You?")

model='llama3.2:3b' temperature=0.8 



#### 출력

In [3]:
print(response.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


또는

In [4]:
# 주피터 이쁘게 출력..? 설치 필요 pprint
from pprint import pprint 
pprint(response.model_dump())

{'additional_kwargs': {},
 'content': "I'm an artificial intelligence model known as Llama. Llama stands "
            'for "Large Language Model Meta AI."',
 'id': 'lc_run--019c2794-bf73-79c2-9647-1bc14e9df69e-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'created_at': '2026-02-04T07:36:34.3867974Z',
                       'done': True,
                       'done_reason': 'stop',
                       'eval_count': 23,
                       'eval_duration': 252446400,
                       'load_duration': 202659800,
                       'logprobs': None,
                       'model': 'llama3.2:3b',
                       'model_name': 'llama3.2:3b',
                       'model_provider': 'ollama',
                       'prompt_eval_count': 31,
                       'prompt_eval_duration': 329151800,
                       'total_duration': 797917500},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_tokens': 31,
                    'out

#### 스트리밍도 가능

In [5]:
for i, chunk in enumerate(model.stream("What is the LangChain")):
    print(chunk.text, end="")


LangChain is an open-source, Python-based framework for building and scaling decentralized applications (dApps) on blockchain networks. It's designed to simplify the process of creating, deploying, and managing smart contracts, as well as interacting with external data sources.

LangChain was founded by Zachary Crouch, a former contributor to OpenZeppelin, in 2022. The framework aims to provide a unified interface for interacting with various blockchain platforms, including Ethereum, Binance Smart Chain, and Solana.

Key Features of LangChain:

1. **Unified API**: LangChain provides a single, standardized API for interacting with different blockchain platforms. This makes it easier for developers to build applications that can run on multiple blockchains.
2. **Smart Contract Management**: LangChain allows developers to create, deploy, and manage smart contracts using a simplified interface. It also provides tools for optimizing contract performance and debugging.
3. **External Data Sou

batch 로 여러개 던지는 것도 가능 - (시간 증가)

In [6]:
inputs = [
    "what is the overfitting?",
    "AI Agent 기반 서비스는 뭐가 있나요?",
    "What's your name?"
]

responses = model.batch(inputs)

for response in responses:
    print(response)

content="Overfitting is a common problem in machine learning and artificial intelligence where a model becomes too specialized to the training data and fails to generalize well to new, unseen data. In other words, a model that is too complex or has too many parameters may fit the noise and irregularities of the training data, but will not be able to reproduce these patterns on new data.\n\nWhen a model overfits, it means that:\n\n1. **The model performs very well on the training data**: The model is able to fit the noise and irregularities of the training data perfectly, resulting in high accuracy.\n2. **The model performs poorly on new data**: When presented with new data that is similar but not identical to the training data, the model's performance drops significantly.\n\nOverfitting can occur due to various reasons such as:\n\n1. **Model complexity**: A model that has too many parameters or is too complex may be prone to overfitting.\n2. **Limited training data**: If the amount of 

In [7]:
for response in responses:
    pprint(response.model_dump())    

{'additional_kwargs': {},
 'content': 'Overfitting is a common problem in machine learning and '
            'artificial intelligence where a model becomes too specialized to '
            'the training data and fails to generalize well to new, unseen '
            'data. In other words, a model that is too complex or has too many '
            'parameters may fit the noise and irregularities of the training '
            'data, but will not be able to reproduce these patterns on new '
            'data.\n'
            '\n'
            'When a model overfits, it means that:\n'
            '\n'
            '1. **The model performs very well on the training data**: The '
            'model is able to fit the noise and irregularities of the training '
            'data perfectly, resulting in high accuracy.\n'
            '2. **The model performs poorly on new data**: When presented with '
            'new data that is similar but not identical to the training data, '
            "the mod

---

#### 스키마 던져주기 (with_structured_output)

In [8]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """영화 정보 스키마"""
    title: str = Field(description="Movie 제목")
    year: int = Field(description="개봉 year")
    director: str = Field(description="director 이름")
    rating: float = Field(description="Move rating (10점 만점)")

In [9]:
model = init_chat_model(
    model="llama3.2:3b", 
    model_provider="ollama",
    temperature=0.8,
)

model_with_structure = model.with_structured_output(Movie)

In [10]:
model_with_structure.invoke("Can you tell me about the movie 'Inception'?")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [11]:
movie_response = model.invoke("Can you tell me about the movie 'Inception'?")
pprint(movie_response.model_dump())    

{'additional_kwargs': {},
 'content': '"Inception" is a 2010 science fiction action film directed by '
            'Christopher Nolan. The movie follows Cobb (played by Leonardo '
            "DiCaprio), a skilled thief who specializes in entering people's "
            'dreams and stealing their secrets.\n'
            '\n'
            'The story takes place while Cobb is offered a chance to redeem '
            'himself and clear his name after being accused of murdering his '
            'wife, Mal (played by Marion Cotillard). Mal is revealed to be the '
            'architect of the dream-sharing technology that Cobb uses for his '
            "heists. The team is approached with a plan by Cobb's former "
            'employer, Arthur (played by Joseph Gordon-Levitt), to plant an '
            "idea in someone's mind instead of stealing one.\n"
            '\n'
            'The concept revolves around entering multiple levels of dreams '
            'within dreams (often referred 

#### Json schema 도 사용 가능 하지만
#### Type Hint 필요함

In [12]:
import json

json_schema = {
    "title": "Movie",
    "description": "A movie with details",
    "type": "object",
    "properties": {
        "title": {
            "type": "string",
            "description": "The title of the movie"
        },
        "year": {
            "type": "integer",
            "description": "The year the movie was released"
        },
        "director": {
            "type": "string",
            "description": "The director of the movie"
        },
        "rating": {
            "type": "number",
            "description": "The movie's rating out of 10"
        }
    },
    "required": ["title", "year", "director", "rating"]
}

In [13]:
model_with_json_schema = model.with_structured_output(json_schema)
model_with_json_schema.invoke("Can you tell me about the movie 'Inception'?")

{'title': 'Inception',
 'year': 2010,
 'director': 'Christopher Nolan',
 'rating': 8.6}